## Aruco标记

ArUco标记是可用于摄像机姿态估计的二进制方形基准标记。它的主要优点是检测简单、快速，并且具有很强的鲁棒性。ArUco 标记是由宽黑色边框和确定其标识符（id）的内部二进制矩阵组成的正方形标记

Aruco有以下三个属性：

- Dictionary（词典）: 定义了一组预定义的Aruco标记集合，不同的词典包含不同数量和类型的标记，每个词典中所有的Aruco标记均包含相同数量的块或位
  - 一个nxn的Aruco标记包含nxn个黑白块（除去一圈黑色边框），常见的有4x4、5x5、6x6、7x7等
- Marker ID（标识符ID）:  每个Aruco标记都有一个唯一的整数ID，用于区分不同的标记，不同的id有不同的黑白块排列
- Marker size, mm（标记尺寸，毫米）： 定义了Aruco标记的物理尺寸

使用aruco.getPredefinedDictionary()函数可以获取预定义的Aruco字典：

函数原型：

`cv2.aruco.getPredefinedDictionary(dictionaryId)`

用来获取预定义的Aruco字典

参数说明：

- dictionaryId: 预定义字典的ID，可以是以下之一：
  - cv2.aruco.DICT_4X4_50
  - cv2.aruco.DICT_4X4_100
  - cv2.aruco.DICT_5X5_100
  - cv2.aruco.DICT_5X5_200
  - cv2.aruco.DICT_6X6_250
  - cv2.aruco.DICT_6X6_500
  - cv2.aruco.DICT_7X7_1000

返回值：

- dictionary: 返回对应的Aruco字典对象

如果报错 `AttributeError: module 'cv2' has no attribute 'aruco'`，说明你的OpenCV版本不支持Aruco模块。你需要安装包含contrib模块的OpenCV版本，可以使用以下命令：

```bash
pip install opencv-contrib-python
# or
conda install -c conda-forge opencv
```

## Aruco标记生成

函数原型：

```python
cv2.aruco.generateImageMarker(dictionary, id, sidePixels[, img[, borderBits]])
```

参数说明：

- `dictionary`：Aruco标记词典对象，可以使用`cv2.aruco.getPredefinedDictionary()`获取
- `id`：要生成的Aruco标记的ID
- `sidePixels`：标记的边长（以像素为单位）
- `img`：可选参数，指定输出图像，如果不提供则创建一个新的图像
- `borderBits`：可选参数，指定标记边框的宽度，默认为1

In [1]:
import cv2
from my_function import imshow

aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
aruco_5x5_id0=cv2.aruco.generateImageMarker(aruco_dict,0,300)
imshow(aruco_5x5_id0=aruco_5x5_id0)

## ChArUco标记板

也可以使用opencv生成ChArUco标记板：

![](./images/ChArUco.png)

初始化类：

```python
board = cv2.aruco.CharucoBoard((squares_x, squares_y), square_length, marker_length, aruco_dict)
```

参数说明：

- `squares_x`：标记板中水平方向的方格数量
- `squares_y`：标记板中垂直方向的方格数量
- `square_length`：每个方格的边长（以米为单位）
- `marker_length`：每个Aruco标记的边长（以米为单位）
- `aruco_dict`：Aruco标记词典对象，可以使用`cv2.aruco.getPredefinedDictionary()`获取

使用`cv2.aruco.CharucoBoard.generateImage()`函数生成ChArUco标记板图像：

函数原型：

```python
board.generateImage((image_width, int(image_width * squares_y / squares_x)), marginSize=margin_size)
```

参数说明：

- `image_width`：生成图像的宽度（以像素为单位）
- `squares_x`：标记板中水平方向的方格数量
- `squares_y`：标记板中垂直方向的方格数量
- `margin_size`：图像边缘的空白区域大小（以像素为单位），默认为0


In [2]:
# 设置ChArUco板参数
squares_x = 7           # X方向方格数
squares_y = 5           # Y方向方格数
square_length = 0.03    # 方格实际长度（单位，米）
marker_length = 0.015   # 标记实际长度（单位，米）
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250) # ArUco字典
image_width = 640       # 输出图像宽度（像素）
margin_size = 20        # 图像边距（像素）

board = cv2.aruco.CharucoBoard(
    (squares_x, squares_y), square_length, marker_length, dictionary)
img = board.generateImage(
    (image_width, int(image_width * squares_y / squares_x)), marginSize=margin_size)
imshow(img=img)

## Aruco标记检测

使用Aruco检测器可以检测图像中的Aruco标记。首先需要创建Aruco检测参数对象和Aruco检测器对象，然后使用检测器的detectMarkers()方法进行检测

`DetectorParameters()`用来创建Aruco检测参数对象，可以通过修改该对象的属性来调整检测行为

函数原型：

`cv2.aruco.DetectorParameters()`

函数原型：

`cv2.aruco.ArucoDetector(aruco_dict, parameters)`

用来创建Aruco检测器对象

参数说明：

- aruco_dict: 通过 `cv2.aruco.getPredefinedDictionary()` 获取的Aruco字典对象
- parameters: 通过 `cv2.aruco.DetectorParameters()` 创建的Aruco检测参数对象

生成检测器后，可以使用其 `detectMarkers()` 方法来检测图像中的Aruco标记

函数原型：

`cv2.detector.detectMarkers(image, dictionary, parameters=None, cameraMatrix=None, distCoeffs=None)`

用来检测图像中的Aruco标记

参数说明：

- image: 输入图像，类型: np.ndarray

- dictionary: 通过 `cv2.aruco.getPredefinedDictionary()` 获取的Aruco字典对象
- parameters: 可选的Aruco检测参数对象
- cameraMatrix: 可选的相机内参矩阵
- distCoeffs: 可选的相机畸变系数

返回值：

- corners: 检测到的Aruco标记的角点列表。类型: list of np.ndarray, 每个元素形状为 (4, 1, 2)，默认顺序为左上、右上、右下、左下
- ids: 检测到的Aruco标记的ID列表。类型: np.ndarray, 形状为 (N, 1)，如果没有检测到标记则为 None
- rejectedImgPoints: 被拒绝的候选标记角点列表。类型: list of np.ndarray

从而实现下面的函数：

In [ ]:
def detect_aruco_corners(frame, aruco_dict):
    """
    检测图像中的ArUco标记
    Args:
        frame: 输入图像帧
        aruco_dict: ArUco字典类型
    Returns:
        corners: 四个角点的像素坐标，形状为(1,4,2)的numpy数组
                 顺序为：左上、右上、右下、左下
                 如果未检测到则返回None
        ids:ID列表
    """
    # 获取ArUco字典
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict)
    # 创建ArUco检测参数
    parameters = cv2.aruco.DetectorParameters()
    # 创建检测器
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)
    # 检测ArUco标记
    corners, ids, _ = detector.detectMarkers(frame)
    # 如果检测到标记，返回标记的四个角点
    if len(corners) > 0:
        return corners,ids
    return None

## Aruco标记绘制

检测到Aruco的corners和ids后，可以使用`cv2.aruco.drawDetectedMarkers()`函数在图像上绘制检测到的Aruco标记

函数原型：

`cv2.aruco.drawDetectedMarkers(image, corners, ids=None, borderColor=(0, 255, 0))`

参数说明：

- `image`：输入图像，在该图像上绘制Aruco标记
- `corners`：检测到的Aruco标记的角点列表，期望格式为list of np.ndarray，每个元素形状为(4, 1, 2)
- `ids`：检测到的Aruco标记的ID列表，期望格式为np.ndarray，形状为(N, 1)
  - 如果只检测单个Aruco标记，就无需传入ids参数
- `borderColor`：可选参数，指定绘制边框的颜色，默认为绿色 (0, 255, 0)

In [4]:
def draw_aruco_markers(frame, corners, ids):
    """
    在图像上绘制检测到的ArUco标记
    Args:
        frame: 输入图像帧
        corners: ArUco标记的角点列表
        ids: ArUco标记的ID列表
    Returns:
        frame: 绘制好的图像帧
    """
    if corners is not None: 
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
    return frame

In [5]:
img=cv2.imread("./images/Arucos.jpg")
corners,ids=detect_aruco_corners(img,cv2.aruco.DICT_6X6_100)
draw_aruco_markers(img, corners, ids)
imshow(img=img)

有的时候我们并不想获得Aruco的轮廓，只想获得Aruco的外接矩形，可以使用`cv2.boundingRect()`函数来实现

> boundingRect()函数详情参阅`图像轮廓检测.ipynb`

In [6]:
def draw_aruco_bounding_boxes(image, corners, ids):
    """
    在图像上绘制ArUco标记的轴对齐边界矩形
    参数说明：
    - `image`: 输入图像
    - `corners`: 检测到的Aruco标记的角点列表
    - `ids`: 检测到的Aruco标记的ID列表
    """
    if ids is not None:
        for i, corner in enumerate(corners):
            # 获取标记的角点
            corner_points = corner[0].astype(np.int32)
            # 计算轴对齐的边界矩形
            x, y, w, h = cv2.boundingRect(corner_points)
            # 绘制外接矩形
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
# 调用示例
import numpy as np

img=cv2.imread("./images/Arucos.jpg")
corners,ids=detect_aruco_corners(img,cv2.aruco.DICT_6X6_100)
draw_aruco_bounding_boxes(img, corners, ids)
imshow(img=img)